In [ ]:
import pandas as pd

In [ ]:
df2 = pd.read_excel("LAhousing.xlsx")

In [ ]:
# figuring out nature of response for one example
# outer response is for geo-encoding using coordinates(LAT,LON)
# then state,county,tract,block info is extacted from response
# which is in turn used to get demographic info from decennial sf1 dataset
url = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x={}&y={}&benchmark=Public_AR_Census2020&vintage=Census2010_Census2020&format=json".format(df2.iloc[3]['CENTER_LON'],df2.iloc[3]['CENTER_LAT'])
try:
    response = requests.get(url)
    response.raise_for_status()
    # access JSOn content
    jsonResponse = response.json()
    print("JSON response")
    #print(jsonResponse)
    print(jsonResponse['result']['geographies']["States"][0]["STATE"])
    print(jsonResponse['result']['geographies']["Counties"][0]["COUNTY"])
    print(jsonResponse['result']['geographies']["Census Tracts"][0]["TRACT"])
    print(jsonResponse['result']['geographies']["Census Blocks"][0]["BLOCK"])
    #extract info om state,county,tract,block
    a=jsonResponse['result']['geographies']["States"][0]["STATE"]
    b=jsonResponse['result']['geographies']["Counties"][0]["COUNTY"]
    c=jsonResponse['result']['geographies']["Census Tracts"][0]["TRACT"]
    d=jsonResponse['result']['geographies']["Census Blocks"][0]["BLOCK"]
    ur = "https://api.census.gov/data/2010/dec/sf1?get=H001001,H010001,H011001,H003002,H003003,H012001,H012002,H012003,H013001,H002002,H002005,P001001,P016003,P013001,P017001,P002002,P002005,P035001,P037001&for=block:{}&in=state:{}&in=county:{}&in=tract:{}&key=90b6f9f89c549fc2bbcd126c0ba7227937d41d9c".format(d,a,b,c)
    try:
        # get demographic info
        res = requests.get(ur)
        res.raise_for_status()
        # access JSOn content
        jsonRes = res.json()
        print(jsonRes)
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')
except HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')
except Exception as err:
    print(f'Other error occurred: {err}')
    

In [ ]:
# repeat for all data points
l = []
for i in range(len(df2)):
    url = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x={}&y={}&benchmark=Public_AR_Census2020&vintage=Census2010_Census2020&format=json".format(df2.iloc[i]['CENTER_LON'],df2.iloc[i]['CENTER_LAT'])
    try:
        response = requests.get(url)
        response.raise_for_status()
        jsonResponse = response.json()
        print(i)
        #print(jsonResponse['result']['geographies']["States"][0]["STATE"])
        #print(jsonResponse['result']['geographies']["Counties"][0]["COUNTY"])
        #print(jsonResponse['result']['geographies']["Census Tracts"][0]["TRACT"])
        #print(jsonResponse['result']['geographies']["Census Blocks"][0]["BLOCK"])
        a=jsonResponse['result']['geographies']["States"][0]["STATE"]
        b=jsonResponse['result']['geographies']["Counties"][0]["COUNTY"]
        c=jsonResponse['result']['geographies']["Census Tracts"][0]["TRACT"]
        d=jsonResponse['result']['geographies']["Census Blocks"][0]["BLOCK"]
        
        # ask for values of different demographic variables
        ur = "https://api.census.gov/data/2010/dec/sf1?get=H001001,H010001,H011001,H003002,H003003,H012001,H012002,H012003,H013001,H002002,H002005,P001001,P016003,P013001,P017001,P002002,P002005,P035001,P037001&for=block:{}&in=state:{}&in=county:{}&in=tract:{}&key=90b6f9f89c549fc2bbcd126c0ba7227937d41d9c".format(d,a,b,c)
        try:
            res = requests.get(ur)
            res.raise_for_status()
            jsonRes = res.json()
            l.append(jsonRes[1][:19])
            #print(jsonRes[1][:19])
        except HTTPError as http_err:
            print(f'HTTP error occurred2: {http_err}')
            l.append([None]*19)
        except Exception as err:
            print(f'Other error occurred2: {err}')
            l.append([None]*19)
    except HTTPError as http_err:
        print(f'HTTP error occurred1: {http_err}')
    except Exception as err:
        print(f'Other error occurred1: {err}') 

dfs = pd.DataFrame(l)

In [ ]:
#Save result in a file
dfs.to_csv("Demography_data.csv",header=False,index=False)